In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:
sentences = []
with open("../data/train_data.txt", "r", encoding="utf-8") as file:
    sentences = file.readlines()

In [53]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

{'de': 1, '0': 2, 'el': 3, 'la': 4, 'en': 5, '1': 6, 'se': 7, 'y': 8, 'a': 9, 'para': 10, 'del': 11, 'los': 12, '2': 13, '3': 14, 'que': 15, 'un': 16, 'las': 17, 'sistema': 18, 'caso': 19, 'una': 20, 'por': 21, 'potencia': 22, 'costo': 23, 'al': 24, 'predespacho': 25, '4': 26, 'g': 27, '6': 28, 'regional': 29, 'datos': 30, 'carga': 31, 'no': 32, 'como': 33, 'barra': 34, 'es': 35, 'resultados': 36, 'demanda': 37, '5': 38, 'i': 39, 'frecuencia': 40, 'con': 41, 'energía': 42, 'cada': 43, '7': 44, 'planta': 45, '9': 46, 'este': 47, 'mw': 48, 'área': 49, '8': 50, 'generadores': 51, 'fig': 52, 'tabla': 53, 'nodo': 54, 'generación': 55, 'mediante': 56, 'su': 57, 'mercado': 58, 'ofertas': 59, 'donde': 60, 'líneas': 61, 'precio': 62, 'o': 63, 'entre': 64, 'mer': 65, 'cual': 66, 'b': 67, 'línea': 68, 'retiro': 69, 'beneﬁcio': 70, 'función': 71, 'puede': 72, 'barras': 73, 'pu': 74, 'transmisión': 75, 'figura': 76, 'generador': 77, 'futuro': 78, 'valor': 79, 'interconexión': 80, 'ser': 81, 'nacion

In [56]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences = pad_sequences(sequences, padding='post')
sequences

array([[ 530,  483,  484, ...,    0,    0,    0],
       [1333,    1,  298, ...,    0,    0,    0],
       [ 592,    1, 1335, ...,    0,    0,    0],
       ...,
       [  35, 1324,   21, ...,    0,    0,    0],
       [   3,   70,   88, ...,    0,    0,    0],
       [1925,  128,  276, ...,    0,    0,    0]])